In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import math

# 거주인구 데이터 불러오기

In [ ]:
df0 = pd.read_csv('/content/drive/MyDrive/세미2차/03_거주인구_RESIDENT_POP.CSV', encoding='cp949')
df0

,base_year,base_month,city,emd_cd,emd_nm,sex,resid_reg_pop,foreign_pop,total_pop
0,2018,1,제주시,50110590,건입동,남성,5085,146,5231
1,2018,1,제주시,50110590,건입동,여성,4715,82,4797
2,2018,1,제주시,50110256,구좌읍,남성,7965,368,8333
3,2018,1,제주시,50110256,구좌읍,여성,7609,184,7793
4,2018,1,서귀포시,50130253,남원읍,남성,9806,428,10234
...,...,...,...,...,...,...,...,...,...
3607,2021,6,제주시,50110250,한림읍,여성,10341,1140,11481
3608,2021,6,제주시,50110600,화북동,남성,12062,169,12231
3609,2021,6,제주시,50110600,화북동,여성,12238,161,12399
3610,2021,6,서귀포시,50130550,효돈동,남성,2660,39,2699


# 데이터 전처리
- region : 6개지역 값 생성(제주시\_중, 서귀포\_중, 제주_동(제주시\_동+서귀포시\_동), 제주\_서(제주시\_서+서귀포시\_서))

- sex : sum으로 남녀통합

- total\_pop은 resid\_reg_pop수치와 매우 유사 -> total 컬럼 제거

# 1. total\_pop 컬럼 제거
외국인인구는 총 인구의 약 10% 이하이며 주민등록 거주인구는 총 인구의 약 90% 이상의 비율이다. 따라서 제거.(EDA코드 중 상관관계 수치 보고 파악)

In [ ]:
df0 = df0.drop('total_pop', axis=1)
df0

,base_year,base_month,city,emd_cd,emd_nm,sex,resid_reg_pop,foreign_pop
0,2018,1,제주시,50110590,건입동,남성,5085,146
1,2018,1,제주시,50110590,건입동,여성,4715,82
2,2018,1,제주시,50110256,구좌읍,남성,7965,368
3,2018,1,제주시,50110256,구좌읍,여성,7609,184
4,2018,1,서귀포시,50130253,남원읍,남성,9806,428
...,...,...,...,...,...,...,...,...
3607,2021,6,제주시,50110250,한림읍,여성,10341,1140
3608,2021,6,제주시,50110600,화북동,남성,12062,169
3609,2021,6,제주시,50110600,화북동,여성,12238,161
3610,2021,6,서귀포시,50130550,효돈동,남성,2660,39


# 2. sex 집계함수 sum

In [ ]:
df1 = df0.groupby(['base_year','base_month', 'city', 'emd_cd', 'emd_nm']).sum()

In [ ]:
df1 = df1.reset_index()

In [ ]:
df1['emd_nm'].unique()

array(['대정읍', '남원읍', '성산읍', '안덕면', '표선면', '송산동', '정방동', '중앙동', '천지동',
       '효돈동', '영천동', '동홍동', '서홍동', '대륜동', '대천동', '중문동', '예래동', '한림읍',
       '애월읍', '구좌읍', '조천읍', '한경면', '추자면', '우도면', '일도1동', '일도2동', '이도1동',
       '이도2동', '삼도1동', '삼도2동', '용담1동', '용담2동', '건입동', '화북동', '삼양동', '봉개동',
       '아라동', '오라동', '연동', '노형동', '외도동', '이호동', '도두동'], dtype=object)

In [ ]:
df1

,base_year,base_month,city,emd_cd,emd_nm,resid_reg_pop,foreign_pop
0,2018,1,서귀포시,50130250,대정읍,21085,1518
1,2018,1,서귀포시,50130253,남원읍,19112,628
2,2018,1,서귀포시,50130259,성산읍,15338,1575
3,2018,1,서귀포시,50130310,안덕면,11462,522
4,2018,1,서귀포시,50130320,표선면,12253,409
...,...,...,...,...,...,...,...
1801,2021,6,제주시,50110650,연동,41910,1875
1802,2021,6,제주시,50110660,노형동,55385,2088
1803,2021,6,제주시,50110670,외도동,22086,274
1804,2021,6,제주시,50110680,이호동,4520,155


# 6개 지역구 만들기
- 서귀포동: 성산읍,표선면,남원읍

- 서귀포중: 대륜동,대천동,동홍동,서홍동,송산동,영천동,예래동,정방동,중문동,중앙동,천지동,효돈동

- 서귀포서: 대정읍,안덕면

- 제주시동:  구좌읍.조천읍

- 제주시중: 건입동,노형동,도두동,봉개동,삼도2동,삼도1동,삼양동,아라동,연동,오라동,외도동,용담2동,용담1동,이도2동,이도1동,이호동,일도2동,일도1동,화북동

- 제주시서: 애월읍,한림읍,한경면

In [ ]:
def test(x) :
  if x in ['성산읍','표선면','남원읍'] : return '서귀포시_동'
  elif x in ['대륜동','대천동','동홍동','서홍동','송산동','영천동','예래동','정방동','중문동','중앙동','천지동','효돈동'] : return '서귀포시_중'
  elif x in ['대정읍','안덕면'] : return '서귀포시_서'
  elif x in ['구좌읍','조천읍'] : return '제주시_동'
  elif x in ['건입동','노형동','도두동','봉개동','삼도2동','삼도1동','삼양동','아라동','연동','오라동','외도동','용담2동','용담1동','이도2동','이도1동','이호동','일도2동','일도1동','화북동'] : return '제주시_중'
  elif x in ['애월읍','한림읍','한경면'] : return '제주시_서'

In [ ]:
df1['region'] = df1['emd_nm'].apply(test)
df1

,base_year,base_month,city,emd_cd,emd_nm,resid_reg_pop,foreign_pop,region
0,2018,1,서귀포시,50130250,대정읍,21085,1518,서귀포시_서
1,2018,1,서귀포시,50130253,남원읍,19112,628,서귀포시_동
2,2018,1,서귀포시,50130259,성산읍,15338,1575,서귀포시_동
3,2018,1,서귀포시,50130310,안덕면,11462,522,서귀포시_서
4,2018,1,서귀포시,50130320,표선면,12253,409,서귀포시_동
...,...,...,...,...,...,...,...,...
1801,2021,6,제주시,50110650,연동,41910,1875,제주시_중
1802,2021,6,제주시,50110660,노형동,55385,2088,제주시_중
1803,2021,6,제주시,50110670,외도동,22086,274,제주시_중
1804,2021,6,제주시,50110680,이호동,4520,155,제주시_중


# 내국인유동인구 데이터병합용 전처리 파일 불러오기

In [ ]:
final_native = pd.read_csv('/content/drive/MyDrive/세미2차/newdata2_내국인.csv', encoding='cp949')
final_native

,base_date,city,emd_nm,n_resd_12,n_work_12,n_visit_12,n_resd_24,n_work_24,n_visit_24,region
0,2018-01-01,서귀포시,남원읍,12912.5789,968.7152,7425.8300,13654.8669,443.0476,3467.5591,서귀포시_동
1,2018-01-01,서귀포시,대륜동,12783.8106,856.8259,6830.8767,14983.1014,904.9435,6174.4550,서귀포시_중
2,2018-01-01,서귀포시,대정읍,16203.3285,689.0849,5951.4391,16962.7390,560.3398,3600.1909,서귀포시_서
3,2018-01-01,서귀포시,대천동,7691.9621,426.2940,4951.4995,9145.0257,416.9276,4597.0410,서귀포시_중
4,2018-01-01,서귀포시,동홍동,17843.2343,1291.0408,6481.1888,20245.8723,1321.7003,6746.1072,서귀포시_중
...,...,...,...,...,...,...,...,...,...,...
54906,2021-06-30,제주시,조천읍,23022.7092,3488.1647,8288.5175,23488.8250,1412.5869,4575.0079,제주시_동
54907,2021-06-30,제주시,추자면,1384.6510,64.5097,548.6349,1369.1858,64.5335,482.2621,NaN
54908,2021-06-30,제주시,한경면,8061.7471,1113.2716,3129.4562,8200.4944,442.7851,1752.7599,제주시_서
54909,2021-06-30,제주시,한림읍,21470.5874,3356.1123,7913.3725,21592.7967,1362.5868,3800.1223,제주시_서


## 거주인구와 join하기 위해 year, month 생성

In [ ]:
final_native['base_date'] = pd.to_datetime(final_native['base_date'])
final_native['base_year'] = final_native['base_date'].dt.year
final_native['base_month'] = final_native['base_date'].dt.month

In [ ]:
final_native

,base_date,city,emd_nm,n_resd_12,n_work_12,n_visit_12,n_resd_24,n_work_24,n_visit_24,region,base_year,base_month
0,2018-01-01,서귀포시,남원읍,12912.5789,968.7152,7425.8300,13654.8669,443.0476,3467.5591,서귀포시_동,2018,1
1,2018-01-01,서귀포시,대륜동,12783.8106,856.8259,6830.8767,14983.1014,904.9435,6174.4550,서귀포시_중,2018,1
2,2018-01-01,서귀포시,대정읍,16203.3285,689.0849,5951.4391,16962.7390,560.3398,3600.1909,서귀포시_서,2018,1
3,2018-01-01,서귀포시,대천동,7691.9621,426.2940,4951.4995,9145.0257,416.9276,4597.0410,서귀포시_중,2018,1
4,2018-01-01,서귀포시,동홍동,17843.2343,1291.0408,6481.1888,20245.8723,1321.7003,6746.1072,서귀포시_중,2018,1
...,...,...,...,...,...,...,...,...,...,...,...,...
54906,2021-06-30,제주시,조천읍,23022.7092,3488.1647,8288.5175,23488.8250,1412.5869,4575.0079,제주시_동,2021,6
54907,2021-06-30,제주시,추자면,1384.6510,64.5097,548.6349,1369.1858,64.5335,482.2621,NaN,2021,6
54908,2021-06-30,제주시,한경면,8061.7471,1113.2716,3129.4562,8200.4944,442.7851,1752.7599,제주시_서,2021,6
54909,2021-06-30,제주시,한림읍,21470.5874,3356.1123,7913.3725,21592.7967,1362.5868,3800.1223,제주시_서,2021,6


# 같은 년도, 월 기준으로 merge 진행
: 거주인구는 월단위 측정이므로, 2018-01-01\~2018-01-31까지 resid\_resd\_pop과 foreign\_pop 값이 동일하게 들어감

In [ ]:
df1

,base_year,base_month,city,emd_cd,emd_nm,resid_reg_pop,foreign_pop,region
0,2018,1,서귀포시,50130250,대정읍,21085,1518,서귀포시_서
1,2018,1,서귀포시,50130253,남원읍,19112,628,서귀포시_동
2,2018,1,서귀포시,50130259,성산읍,15338,1575,서귀포시_동
3,2018,1,서귀포시,50130310,안덕면,11462,522,서귀포시_서
4,2018,1,서귀포시,50130320,표선면,12253,409,서귀포시_동
...,...,...,...,...,...,...,...,...
1801,2021,6,제주시,50110650,연동,41910,1875,제주시_중
1802,2021,6,제주시,50110660,노형동,55385,2088,제주시_중
1803,2021,6,제주시,50110670,외도동,22086,274,제주시_중
1804,2021,6,제주시,50110680,이호동,4520,155,제주시_중


In [ ]:
n_resid_merge = pd.merge(final_native, df1, left_on=['base_year', 'base_month', 'city', 'emd_nm'], 
                         right_on=['base_year', 'base_month', 'city', 'emd_nm'], how='outer')
n_resid_merge

,base_date,city,emd_nm,n_resd_12,n_work_12,n_visit_12,n_resd_24,n_work_24,n_visit_24,region_x,base_year,base_month,emd_cd,resid_reg_pop,foreign_pop,region_y
0,2018-01-01,서귀포시,남원읍,12912.5789,968.7152,7425.8300,13654.8669,443.0476,3467.5591,서귀포시_동,2018,1,50130253,19112,628,서귀포시_동
1,2018-01-02,서귀포시,남원읍,13644.7718,1823.9205,5506.4173,13734.3061,759.4897,2443.2286,서귀포시_동,2018,1,50130253,19112,628,서귀포시_동
2,2018-01-03,서귀포시,남원읍,13515.1857,1854.2316,6094.3738,13614.5918,681.6981,2557.9769,서귀포시_동,2018,1,50130253,19112,628,서귀포시_동
3,2018-01-04,서귀포시,남원읍,12644.7535,1784.2432,5572.5483,13435.1816,692.6310,2669.9062,서귀포시_동,2018,1,50130253,19112,628,서귀포시_동
4,2018-01-05,서귀포시,남원읍,13389.4337,1685.4956,4564.8582,13448.5294,638.2652,2556.1146,서귀포시_동,2018,1,50130253,19112,628,서귀포시_동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54906,2021-06-26,제주시,화북동,30359.1353,1726.9094,10787.9762,31086.7498,1324.1743,10760.4770,제주시_중,2021,6,50110600,24300,330,제주시_중
54907,2021-06-27,제주시,화북동,29565.1500,1351.0056,10367.1751,31292.3531,1153.9619,10471.0976,제주시_중,2021,6,50110600,24300,330,제주시_중
54908,2021-06-28,제주시,화북동,30894.5045,3580.5841,12110.5064,31653.6407,1926.1140,10968.1114,제주시_중,2021,6,50110600,24300,330,제주시_중
54909,2021-06-29,제주시,화북동,30452.3461,3549.7202,12125.7589,31253.6449,2011.0351,11184.8210,제주시_중,2021,6,50110600,24300,330,제주시_중


In [ ]:
n_resid_merge = n_resid_merge.drop(['base_year', 'base_month', 'emd_cd', 'region_y'], axis=1)
n_resid_merge

,base_date,city,emd_nm,n_resd_12,n_work_12,n_visit_12,n_resd_24,n_work_24,n_visit_24,region_x,resid_reg_pop,foreign_pop
0,2018-01-01,서귀포시,남원읍,12912.5789,968.7152,7425.8300,13654.8669,443.0476,3467.5591,서귀포시_동,19112,628
1,2018-01-02,서귀포시,남원읍,13644.7718,1823.9205,5506.4173,13734.3061,759.4897,2443.2286,서귀포시_동,19112,628
2,2018-01-03,서귀포시,남원읍,13515.1857,1854.2316,6094.3738,13614.5918,681.6981,2557.9769,서귀포시_동,19112,628
3,2018-01-04,서귀포시,남원읍,12644.7535,1784.2432,5572.5483,13435.1816,692.6310,2669.9062,서귀포시_동,19112,628
4,2018-01-05,서귀포시,남원읍,13389.4337,1685.4956,4564.8582,13448.5294,638.2652,2556.1146,서귀포시_동,19112,628
...,...,...,...,...,...,...,...,...,...,...,...,...
54906,2021-06-26,제주시,화북동,30359.1353,1726.9094,10787.9762,31086.7498,1324.1743,10760.4770,제주시_중,24300,330
54907,2021-06-27,제주시,화북동,29565.1500,1351.0056,10367.1751,31292.3531,1153.9619,10471.0976,제주시_중,24300,330
54908,2021-06-28,제주시,화북동,30894.5045,3580.5841,12110.5064,31653.6407,1926.1140,10968.1114,제주시_중,24300,330
54909,2021-06-29,제주시,화북동,30452.3461,3549.7202,12125.7589,31253.6449,2011.0351,11184.8210,제주시_중,24300,330


In [ ]:
n_resid_merge.rename(columns={'region_x':'region'}, inplace=True)

# 내국인 유동인구 & 거주인구 최종 merge 파일 저장

In [ ]:
n_resid_merge.to_csv("/content/drive/MyDrive/세미2차/mergedata2_내국인거주인구.csv", index=False, encoding='CP949')